# DNN

**Reference:** https://github.com/vinayakumarr/Network-Intrusion-Detection

**Dataset:** http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html

In [1]:
import numpy as np
np.random.seed(1337)  # for reproducibility

import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

from utils.preprocess import get_classification_report

from sklearn.preprocessing import Normalizer

In [2]:
DATA = '../data/'
MODELS = '../models/'
TEST = 0.3

In [3]:
traindata = pd.read_csv(DATA+'kddcup.data.csv')
traindata

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.00,0.0,0.00,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.00,0.0,0.00,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.00,0.0,0.00,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.00,0.0,0.00,0.0,0.0,normal.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4898426,0,tcp,http,SF,212,2288,0,0,0,0,...,255,1.0,0.0,0.33,0.05,0.0,0.01,0.0,0.0,normal.
4898427,0,tcp,http,SF,219,236,0,0,0,0,...,255,1.0,0.0,0.25,0.05,0.0,0.01,0.0,0.0,normal.
4898428,0,tcp,http,SF,218,3610,0,0,0,0,...,255,1.0,0.0,0.20,0.05,0.0,0.01,0.0,0.0,normal.
4898429,0,tcp,http,SF,219,1234,0,0,0,0,...,255,1.0,0.0,0.17,0.05,0.0,0.01,0.0,0.0,normal.


In [4]:
traindata.columns

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41'],
      dtype='object')

In [5]:
from utils import preprocess
traindata,testdata=preprocess.preprocess(traindata)

In [6]:
class_to_idx={d:i for i,d in enumerate(traindata['Y'].unique())}
traindata['Y']=traindata['Y'].replace(class_to_idx)
testdata['Y']=testdata['Y'].replace(class_to_idx)

In [7]:
X = traindata.drop(columns=['Y'])
Y = traindata['Y']
C = testdata['Y']
T = testdata.drop(columns=['Y'])

In [8]:
scaler = Normalizer().fit(X)
X_train = scaler.transform(X)
X_test = scaler.transform(T)

In [9]:
y_train= Y
y_test= C

In [10]:
len(y_test.unique())

4

In [11]:
from utils.preprocess import get_classification_report

def build_model(activation):
    model = Sequential()
    model.add(Dense(10, activation=activation, input_shape=X_train.shape[1:]))
    model.add(Dropout(0.5))
    model.add(Dense(len(y_test.unique()), activation="softmax"))
    return model

def train_and_evaluate_model(model, epochs=4, batch_size=512,validation_split=0.25):
    history=model.fit(X_train,y_train,epochs=epochs,batch_size=batch_size, validation_split=validation_split)
    print("Evaluating Model: ")
    model.evaluate(X_test,y_test,batch_size)
    report=get_classification_report(X_test,y_test,model)
    return history, report

In [12]:
relu_model=build_model("relu")
tanh_model=build_model("tanh")

relu_model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",metrics=['accuracy'])
tanh_model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",metrics=['accuracy'])

In [14]:
# train
history,report=train_and_evaluate_model(relu_model)
print(report)


Epoch 1/4
4306/4306 [==============================] - 6s 1ms/step - loss: 0.5071 - accuracy: 0.8363 - val_loss: 0.1537 - val_accuracy: 0.9772
Epoch 2/4
4306/4306 [==============================] - 4s 1ms/step - loss: 0.2925 - accuracy: 0.9203 - val_loss: 0.0990 - val_accuracy: 0.9889
Epoch 3/4
4306/4306 [==============================] - 4s 1ms/step - loss: 0.2381 - accuracy: 0.9347 - val_loss: 0.0809 - val_accuracy: 0.9891
Epoch 4/4
4306/4306 [==============================] - 4s 978us/step - loss: 0.2107 - accuracy: 0.9451 - val_loss: 0.0729 - val_accuracy: 0.9891
Evaluating Model: 
3827/3827 [==============================] - 2s 574us/step - loss: 0.0724 - accuracy: 0.9892


C:\Users\Ashwin\miniconda3\envs\TF_CPU\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ashwin\miniconda3\envs\TF_CPU\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1123155
           1       0.97      1.00      0.99    428807
           2       0.98      1.00      0.99    389112
           3       0.00      0.00      0.00     18299

    accuracy                           0.99   1959373
   macro avg       0.74      0.75      0.74   1959373
weighted avg       0.98      0.99      0.98   1959373



C:\Users\Ashwin\miniconda3\envs\TF_CPU\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
tanh_hist,tanh_report=train_and_evaluate_model(tanh_model)
print(tanh_report)

Epoch 1/4
4306/4306 [==============================] - 5s 1ms/step - loss: 0.3039 - accuracy: 0.9241 - val_loss: 0.0793 - val_accuracy: 0.9820
Epoch 2/4
4306/4306 [==============================] - 4s 963us/step - loss: 0.1230 - accuracy: 0.9738 - val_loss: 0.0594 - val_accuracy: 0.9884
Epoch 3/4
4306/4306 [==============================] - 4s 999us/step - loss: 0.1011 - accuracy: 0.9783 - val_loss: 0.0554 - val_accuracy: 0.9886
Epoch 4/4
4306/4306 [==============================] - 4s 991us/step - loss: 0.0924 - accuracy: 0.9795 - val_loss: 0.0536 - val_accuracy: 0.9888
Evaluating Model: 
3827/3827 [==============================] - 2s 573us/step - loss: 0.0532 - accuracy: 0.9889


C:\Users\Ashwin\miniconda3\envs\TF_CPU\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ashwin\miniconda3\envs\TF_CPU\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1123155
           1       0.97      1.00      0.99    428807
           2       0.98      0.99      0.99    389112
           3       0.00      0.00      0.00     18299

    accuracy                           0.99   1959373
   macro avg       0.74      0.75      0.74   1959373
weighted avg       0.98      0.99      0.98   1959373



C:\Users\Ashwin\miniconda3\envs\TF_CPU\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
y_test.unique()

array([0, 2, 1, 3], dtype=int64)

In [17]:
print(tanh_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   1123155
           1       0.97      1.00      0.99    428807
           2       0.98      0.99      0.99    389112
           3       0.00      0.00      0.00     18299

    accuracy                           0.99   1959373
   macro avg       0.74      0.75      0.74   1959373
weighted avg       0.98      0.99      0.98   1959373



In [18]:
relu_model.save("../models/relu_dnn.hdf5")
tanh_model.save("../models/tanh_dnn.hdf5")